In [9]:
import os
import pickle
from ucimlrepo import fetch_ucirepo

def fetch_students_dropout_data():
    local_file_path = './data/students_dropout_data.pkl'
    
    if not os.path.exists(local_file_path):
        # Fetch dataset from UCIMLRepo
        dataset = fetch_ucirepo(id=697)
        
        if dataset:
            # Save the dataset to a local file using pickle
            with open(local_file_path, 'wb') as file:
                pickle.dump(dataset, file)
            print("Data saved to:", local_file_path)
        else:
            print("Failed to fetch data from UCIMLRepo.")
            return None
    else:
        # Load the dataset from the local file using pickle
        with open(local_file_path, 'rb') as file:
            dataset = pickle.load(file)
    
    return dataset

# Example usage
dataset = fetch_students_dropout_data()
if dataset is not None:
    print("Dataset loaded successfully.")
    print("Features DataFrame head:")
    print(dataset.data.features.shape)
    

Dataset loaded successfully.
Features DataFrame head:
(4424, 36)


In [2]:
predict_students_dropout_and_academic_success.variables

,name,role,type,demographic,description,units,missing_values
0,Marital Status,Feature,Integer,Marital Status,1 – single 2 – married 3 – widower 4 – divorce...,None,no
1,Application mode,Feature,Integer,None,1 - 1st phase - general contingent 2 - Ordinan...,None,no
2,Application order,Feature,Integer,None,Application order (between 0 - first choice; a...,None,no
3,Course,Feature,Integer,None,33 - Biofuel Production Technologies 171 - Ani...,None,no
4,Daytime/evening attendance,Feature,Integer,None,1 – daytime 0 - evening,None,no
5,Previous qualification,Feature,Integer,Education Level,1 - Secondary education 2 - Higher education -...,None,no
6,Previous qualification (grade),Feature,Continuous,None,Grade of previous qualification (between 0 and...,None,no
7,Nacionality,Feature,Integer,Nationality,1 - Portuguese; 2 - German; 6 - Spanish; 11 - ...,None,no
8,Mother's qualification,Feature,Integer,Education Level,1 - Secondary Education - 12th Year of Schooli...,None,no
9,Father's qualification,Feature,Integer,Education Level,1 - Secondary Education - 12th Year of Schooli...,None,no
